In [2]:
# import libraries
import numpy as np


In [3]:
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [4]:
# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [5]:
# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [6]:
# activation function and its derivative
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

In [7]:
# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [8]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [32]:
def generate_data(n, noise=0.5):
    X = np.random.rand(n, 2)
    X_noise = np.random.normal(0, noise, (n, 2))
    X = X + X_noise
    # apply threshold
    X = np.array(X>0.5, dtype=int)
    return X

X = generate_data(10)
# take and
y = np.array(np.logical_and(X[:, 0], X[:, 1]), dtype=int)

In [33]:
X.reshape((10, 1, 2))
y.reshape((10, 1, 1))

array([[[0]],

       [[0]],

       [[0]],

       [[1]],

       [[1]],

       [[1]],

       [[1]],

       [[0]],

       [[0]],

       [[0]]])

In [36]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]], [[0, 1]], [[1, 0]], [[0, 1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]], [[1]], [[1]], [[1]]])
print(x_train.shape, y_train.shape)
# network
net = Network()
net.add(FCLayer(2, 2))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(2, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.15)

# test
out = net.predict(x_train)
print(out)

(7, 1, 2) (7, 1, 1)
epoch 1/1000   error=0.314686
epoch 2/1000   error=0.229840
epoch 3/1000   error=0.225222
epoch 4/1000   error=0.223864
epoch 5/1000   error=0.223141
epoch 6/1000   error=0.222629
epoch 7/1000   error=0.222222
epoch 8/1000   error=0.221882
epoch 9/1000   error=0.221590
epoch 10/1000   error=0.221336
epoch 11/1000   error=0.221112
epoch 12/1000   error=0.220913
epoch 13/1000   error=0.220734
epoch 14/1000   error=0.220571
epoch 15/1000   error=0.220423
epoch 16/1000   error=0.220286
epoch 17/1000   error=0.220160
epoch 18/1000   error=0.220043
epoch 19/1000   error=0.219933
epoch 20/1000   error=0.219830
epoch 21/1000   error=0.219734
epoch 22/1000   error=0.219643
epoch 23/1000   error=0.219557
epoch 24/1000   error=0.219476
epoch 25/1000   error=0.219400
epoch 26/1000   error=0.219327
epoch 27/1000   error=0.219257
epoch 28/1000   error=0.219192
epoch 29/1000   error=0.219129
epoch 30/1000   error=0.219069
epoch 31/1000   error=0.219012
epoch 32/1000   error=0.2189